In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import csv
import numpy as np
import pandas as pd
%matplotlib inline
from pathlib import Path
import scipy.interpolate 
from tqdm.notebook import tqdm
from scipy.optimize import curve_fit
from scipy import integrate
import matplotlib.patches as mpatches
import warnings
warnings.filterwarnings("ignore")
from scipy.linalg import eigh as sp_eigh
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mlxtend.plotting import plot_decision_regions
from sklearn.preprocessing import normalize
from scipy.signal import argrelextrema
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import math
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from mlxtend.plotting import plot_decision_regions

In [ ]:
def make_data(path):
    with open(path, 'r') as f:
        num_cols = len(f.readline().split(sep=","))
        f.seek(0)
        np.genfromtxt(path, delimiter=',',skip_header=2,usecols = range(1,num_cols))
    return np.transpose(np.genfromtxt(path, delimiter=',',skip_header=2,usecols = range(5,num_cols)))
def sample_name(path):
    return path.split(sep="\\")[-1][:-4]
def sample_class(path):
    if "-" in path.split(sep="\\")[-1] or " " in path.split(sep="\\")[-1]:
        return path.split(sep="\\")[-1][:3]
    else:
        return path.split(sep="\\")[-1][0]
def class_filepaths(sp_class):
    class_paths = []
    if type(sp_class) == type("1"):
        for path in filepaths:
            if sample_class(path) == sp_class:
                class_paths.append(path)
        return class_paths
    else:
        for sp in sp_class:
            for path in filepaths:
                if sample_class(path) == sp_class:
                    class_paths.append(path)
        return class_paths
def plot_heat_map(path,a=0.9995,colorbar=True,xlabel=True,ylabel=True,title=True,ax=plt,raman_spline=False,cut=True,
                  EM_wavelenght_start=300,EM_wavelenght_finish=550,EX_wavelenght_start=250,EX_wavelenght_finish=400):
    if raman_spline == True:
        if cut==True:
            data = data_cut(cut_raman_spline_data(normalize_data(make_data(path))))
            data[data >np.quantile(data,a)] = 0
            maxim=np.max(data)
            data = data/maxim
            EM_wavelenghts_cut = EM_wavelenghts[np.where(EM_wavelenghts==EM_wavelenght_start)[0][0]:
               np.where(EM_wavelenghts==EM_wavelenght_finish)[0][0]]
            EX_wavelenghts_cut = EX_wavelenghts[np.where(EX_wavelenghts==EX_wavelenght_start)[0][0]:
               np.where(EX_wavelenghts==EX_wavelenght_finish)[0][0]]
            if ax == plt:
                im = ax.pcolormesh(EM_wavelenghts_cut,EX_wavelenghts_cut,data,shading="gouraud",vmin=0,vmax=1,cmap=plt.get_cmap('rainbow'))
                if xlabel == True:
                    ax.xlabel("EM_wavelenghts")
                if ylabel == True:
                    ax.ylabel("EX_wavelenghts")
                if title == True:
                    ax.title(sample_name(path))
            else:
                im = ax.pcolormesh(EM_wavelenghts_cut,EX_wavelenghts_cut,data,shading="gouraud",vmin=0,vmax=1,cmap=plt.get_cmap('rainbow'))
                if xlabel == True:
                    ax.set_xlabel("EM_wavelenghts")
                if ylabel == True:
                    ax.set_ylabel("EX_wavelenghts")
                if title == True:
                    ax.set_title(sample_name(path))
            return im
        else:
            data = data_cut(cut_raman_spline_data(normalize_data(make_data(path))))
            data[data >np.quantile(data,a)] = 0
            maxim=np.max(data)
            data = data/maxim
    else:
        data = make_data(path)
        data[data >np.quantile(data,a)] = 0
        maxim=np.max(data)
        data = data/maxim
    if ax == plt:
        im = ax.pcolormesh(EM_wavelenghts,EX_wavelenghts,data,shading="gouraud",vmin=0,vmax=1,cmap=plt.get_cmap('rainbow'))
        if xlabel == True:
            ax.xlabel("EM_wavelenghts")
        if ylabel == True:
            ax.ylabel("EX_wavelenghts")
        if title == True:
            ax.title(sample_name(path))
    else:
        im = ax.pcolormesh(EM_wavelenghts,EX_wavelenghts,data,shading="gouraud",vmin=0,vmax=1,cmap=plt.get_cmap('rainbow'))
        if xlabel == True:
            ax.set_xlabel("EM_wavelenghts")
        if ylabel == True:
            ax.set_ylabel("EX_wavelenghts")
        if title == True:
            ax.set_title(sample_name(path))
    return im
def plot_heat_map_class(nums_ax_one_plot=9,nums_col=3,nums_row=3,mode="full"):
    if mode == "full":
        nums_plot=math.ceil(len(filepaths)/nums_ax_one_plot)
        index_sample = 0
        for i in range(nums_plot):
            fig,ax = plt.subplots(nums_row,nums_col,sharex=True,sharey=True,figsize=(18,15))
            for row in range(nums_row):
                ax[row,0].set_ylabel("EX wavelenghts, nm")
                for col in range(nums_col):
                    ax[nums_row-1,col].set_xlabel("EM wavelenghts, nm")
                    if index_sample == len(filepaths):
                        break
                    im = plot_heat_map(filepaths[index_sample],colorbar=False,ax=ax[row,col],xlabel=False,ylabel=False,title=True)
                    index_sample += 1
            fig.colorbar(im, ax=ax)
    else:
        nums_plot=math.ceil(len(class_filepaths(mode))/nums_ax_one_plot)
        index_sample = 0
        for i in range(nums_plot):
            fig,ax = plt.subplots(nums_row,nums_col,sharex=True,sharey=True,figsize=(18,15))
            for row in range(nums_row):
                ax[row,0].set_ylabel("EX wavelenghts, nm")
                for col in range(nums_col):
                    ax[nums_row-1,col].set_xlabel("EM wavelenghts, nm")
                    if index_sample == len(class_filepaths(mode)):
                        break
                    im = plot_heat_map(class_filepaths(mode)[index_sample],colorbar=False,ax=ax[row,col],xlabel=False,ylabel=False,title=True)
                    index_sample += 1
            fig.colorbar(im, ax=ax)                         
def plot_2d(path,xlabel=True,ylabel=True,title=True,mode="full",ex_wave=None):
    data = make_data(path)
    name = sample_name(path)
    if mode == "full":
        for i,ex_wave in enumerate(EX_wavelenghts):
            plt.plot(EM_wavelenghts,data[i])
            if title == True:
                plt.title(f"{name}, {ex_wave} nm")
            if xlabel == True:
                plt.xlabel("EM_wavelenghts")
            if ylabel == True:
                plt.ylabel("Fluorescence")
            plt.show()
    if mode == "single":
        plt.plot(EM_wavelenghts,data[np.where(EX_wavelenghts==ex_wave)[0][0]])
        if title == True:
            plt.title(f"{name}, {ex_wave} nm")
        if xlabel == True:
            plt.xlabel("EM_wavelenghts")
        if ylabel == True:
            plt.ylabel("Fluorescence")
def plot_2d_class(sp_class,xlabel=True,ylabel=True,title=False):
    class_paths = []
    for path in filepaths:
        if sample_class(path) == sp_class:
            class_paths.append(path)
    for i,ex_wave in enumerate(EX_wavelenghts):
        for path in class_paths:
            data = make_data(path)
            name = sample_name(path)
            plt.plot(EM_wavelenghts,data[i],label=name)
        if title == True:
            plt.title(f"{sp_class}, {ex_wave} nm")
        if xlabel == True:
            plt.xlabel("EM_wavelenghts")
        if ylabel == True:
            plt.ylabel("CPS, mA")
        plt.legend()
def normalize_data(data,norm_type="max",a=0.9995):
    data[data >np.quantile(data,a)] = 0
    return normalize(data, axis=0, norm=norm_type)
def where_is_raman(exc_wv,Omega=3430.):
    """
    return the central wavelength of solvent raman scattering band (stokes mode) 
    for given excitation wavelength (exc_wv) in nm
    exc_wv(float) - excitation wavelength in nm
    Omega(float) - molecular oscillations frequency in cm^-1
    return(float) central wavelength of raman stokes mode peak
    """
    return 1e7/(1e7/exc_wv - Omega)
def cut_peak_spline(em_wvs, fl, peak_position, peak_half_width=15.,points_in_spline=10):
    """
    em_wvs - 1d numpy array of emission wavelength
    fl - 1d numpy array of fluorescence
    peak_position (float) - position of peak in nanometers
    peak_half_width - half-width at half-maximum of peak to cut
    """
    mask = ((em_wvs < peak_position - peak_half_width)|(em_wvs > peak_position + peak_half_width))&(~np.isnan(fl))    
    knots = em_wvs[mask][1:-1][::points_in_spline]
    spline = scipy.interpolate.LSQUnivariateSpline(em_wvs[mask],fl[mask],knots,k=2,ext=1)    
    spline_wvs = em_wvs[mask]
    return knots,mask,spline_wvs,spline
def cut_raman_spline(em_wvs,fl,exc_wv,raman_freq=3430.,raman_hwhm=15,points_in_spline=10):
    """
    em_wvs - 1d numpy array of emission wavelength
    fl - 1d numpy array of fluorescence
    raman_freq (float) = 3430. - frequency of raman scattering in cm^-1
    raman_hwhm (float) - raman band half width at half maximum to cut in nm
    points_in_spline(int) - number of points in interpolating spline
    
    return 1d numpy array of fluorescence intensity with cutted emission
    """
    res = cut_peak_spline(em_wvs,fl,
                          peak_position=where_is_raman(exc_wv,Omega=raman_freq),
                          peak_half_width=raman_hwhm, points_in_spline=10)
    return res[-1](em_wvs)
def cut_raman_spline_data(data):
    for i,row in enumerate(data):
        data[i] = cut_raman_spline(EM_wavelenghts,row,EX_wavelenghts[i])
    return data
def nearest(x, c):
    x=np.array(x)
    a=abs(x-c)
    a = a.tolist()
    m=a.index(min(a))
    return m
def epsilon(data):
    return data[:,1][nearest(UV_wavelenghts,254)]
def E2_E3(data):
    return data[:,1][nearest(UV_wavelenghts,265)]/data[:,1][nearest(UV_wavelenghts,365)]
def E4_E6(data):
    return data[:,1][nearest(UV_wavelenghts,465)]/data[:,1][nearest(UV_wavelenghts,665)]
def epsilon(data):
    return data[:,1][nearest(UV_wavelenghts,254)]
def lambda_UV(data):
    index_450 = int(nearest(UV_wavelenghts, 450))
    index_550 = int(nearest(UV_wavelenghts, 550))
    a,b = np.polyfit(UV_wavelenghts[index_550:index_450],np.log(data[:,1][index_550:index_450]),1)
    return 1/abs(a)
def asm_350(data):
    index = np.where(EX_wavelenghts == 350)[0][0]
    row = normalize_data(data)[index]
    spline = cut_raman_spline(EM_wavelenghts,row,350)
    return np.trapz(spline[np.where(EM_wavelenghts == 420)[0][0]:np.where(EM_wavelenghts == 460)[0][0]])/np.trapz(spline[np.where(EM_wavelenghts == 550)[0][0]:np.where(EM_wavelenghts == 600)[0][0]])
def asm_350_class(sp_class):
    list_asm_350 =[]
    for path in class_filepaths(sp_class):
        list_asm_350.append(asm_350(make_data(path)))
    return list_asm_350
def asm_280(data):
    index = np.where(EX_wavelenghts == 280)[0][0]
    row = normalize_data(data)[index]
    spline = cut_raman_spline(EM_wavelenghts,row,280)
    return np.trapz(spline[np.where(EM_wavelenghts == 350)[0][0]:np.where(EM_wavelenghts == 400)[0][0]])/np.trapz(spline[np.where(EM_wavelenghts == 475)[0][0]:np.where(EM_wavelenghts == 535)[0][0]])
def asm_280_class(sp_class):
    list_asm_280 =[]
    for path in class_filepaths(sp_class):
        list_asm_280.append(asm_280(make_data(path)))
    return list_asm_280
def sample_list(sp_class):
    list_sample =[]
    for path in class_filepaths(sp_class):
        list_sample.append(sample_name(path))
    return list_sample
def UV_data(path,norm=False):
    if norm == True:
        data = np.genfromtxt(path, delimiter=',',skip_header=1)
        data[:,1] = (data[:,1] - np.min(data[:,1])) / (np.max(data[:,1]) - np.min(data[:,1]))
        return data
    else:
        return np.genfromtxt(path, delimiter=',',skip_header=1)
def UV_plot(path,xlabel=True,ylabel=True,title=True,ln_scale=False):
    data = UV_recalibrate(UV_data(path))
    sp_name = sample_name(path)
    if ln_scale == False:
        plt.plot(UV_wavelenghts,data[:,1])
        if ylabel == True:
            plt.ylabel("intensity")
    if ln_scale == True:
        plt.plot(UV_wavelenghts,np.log(data[:,1]),label = sp_name)
        if ylabel == True:
            plt.ylabel("ln(intensity)")
    if title == True:
        plt.title(f"{sp_name}")
    if xlabel == True:
        plt.xlabel("Wavelenghts, nm")

def all_class():
    all_class_list=[]
    for path in filepaths:
        all_class_list.append(sample_class(path))
    unique_class = np.unique(np.array(all_class_list))
    return unique_class
def all_sample_names(mode):
    all_sample_list=[]
    if mode == "fluo":
        for path in filepaths:
            all_sample_list.append(sample_name(path))
        return all_sample_list
    if mode == "UV":
        for path in filepaths_UV:
            all_sample_list.append(sample_name(path))
        return all_sample_list
def asm_280_all():
    list_asm_280 =[]
    for path in filepaths:
        list_asm_280.append(asm_280(make_data(path)))
    return list_asm_280
def asm_350_all():
    list_asm_350 =[]
    for path in filepaths:
        list_asm_350.append(asm_350(make_data(path)))
    return list_asm_350
def blue_shift_area(data,mode="one"):
    if mode == "one":
        first_row = data[:5,]
        quadrants_list = []
        coordinate = 0
        for i in range(0,first_row.shape[1],10):
            if i == 0:
                current_max_quadrant = np.sum(first_row[:,i:i+10])
                continue
            if current_max_quadrant<np.sum(first_row[:,i:i+10]):
                current_max_quadrant = np.sum(first_row[:,i:i+10])
                coordinate = i
        blue_shift_quadrant = data[5:10,coordinate-10:coordinate]
        return np.sum(blue_shift_quadrant)
def data_cut(data,EM_wavelenght_start=300,EM_wavelenght_finish=550,EX_wavelenght_start=250,EX_wavelenght_finish=400):
    return data[np.where(EX_wavelenghts==EX_wavelenght_start)[0][0]:
               np.where(EX_wavelenghts==EX_wavelenght_finish)[0][0],
               np.where(EM_wavelenghts==EM_wavelenght_start)[0][0]:
               np.where(EM_wavelenghts==EM_wavelenght_finish)[0][0]]
def UV_recalibrate(data):
    data = data - np.min(data)
    return data  
def max_intense_wavelenghts(data):
    part_data = data[:20,]
    current_max_quadrant_list = []
    for row in range(0,part_data.shape[0],5):
        for column in range(0,part_data.shape[1],10):
            current_max_quadrant_list.append([np.sum(part_data[row:row+5,column:column+5]),row,column])
    current_max_quadrant_array = np.array(current_max_quadrant_list)
    max_intense_quadrant_index = np.where(current_max_quadrant_array[:,0]==np.max(current_max_quadrant_array[:,0]))[0][0]
    EM_wave_max = EM_wavelenghts[int(current_max_quadrant_array[max_intense_quadrant_index][2]+2)]+2.5
    EX_wave_max = EX_wavelenghts[int(current_max_quadrant_array[max_intense_quadrant_index][1]+2)] +2.5
    return EM_wave_max,EX_wave_max
def over_class(path=None, mode="single"):
    if mode=="single":
        if path.split(sep="\\")[-1][0].isdigit():
            return "map"
        else:
            return path.split(sep="\\")[-1][0]
    if mode == "all":
        over_class_list = []
        for path in filepaths:
            if path.split(sep="\\")[-1][0].isdigit():
                over_class_list.append(path.split(sep="\\")[-1][0])
            else:
                over_class_list.append(path.split(sep="\\")[-1][0])
        return over_class_list
def subplots_parametres(data):
    descriptor_list = data.columns
    if len(descriptor_list) < 10:
        num_rows = 3
        num_cols = 3    
    if len(descriptor_list) < 7:
        num_rows = 2
        num_cols = 3
    if len(descriptor_list) < 5:
        num_rows = 2
        num_cols = 2
    return num_rows,num_cols
def dataset_boxplot(data,level_index=0):
    descriptor_list = data.columns
    X = data.values
    y = data.index.get_level_values(level_index).tolist()
    num_rows,num_cols = subplots_parametres(data)
    fig,ax = plt.subplots(num_rows,num_cols,figsize=(20,15))
    descriptor_col_index = 0
    for row in range(num_rows):
        for col in range(num_cols):
            if descriptor_col_index == len(descriptor_list):
                break
            sns.boxplot(y=X[:,descriptor_col_index],ax=ax[row,col],hue=y)
            ax[row,col].set_ylabel(f"{descriptor_list[descriptor_col_index]}")
            ax[row,col].set_title(f"Значения {descriptor_list[descriptor_col_index]}")
            descriptor_col_index +=1
def dataset_swarmplot(data,level_index=0):
    descriptor_list = data.columns
    X = data.values
    y = data.index.get_level_values(level_index).tolist()
    num_rows,num_cols = subplots_parametres(data)
    fig,ax = plt.subplots(num_rows,num_cols,figsize=(20,15))
    descriptor_col_index = 0
    for row in range(num_rows):
        for col in range(num_cols):
            if descriptor_col_index == len(descriptor_list):
                break
            sns.swarmplot(y=X[:,descriptor_col_index],ax=ax[row,col],hue=y)
            ax[row,col].set_ylabel(f"{descriptor_list[descriptor_col_index]}")
            ax[row,col].set_title(f"Значения {descriptor_list[descriptor_col_index]}")
            descriptor_col_index +=1
def dataset_violinplot(data,level_index=0):
    descriptor_list = data.columns
    X = data.values
    y = data.index.get_level_values(level_index).tolist()
    num_rows,num_cols = subplots_parametres(data)
    fig,ax = plt.subplots(num_rows,num_cols,figsize=(20,15))
    descriptor_col_index = 0
    for row in range(num_rows):
        for col in range(num_cols):
            if descriptor_col_index == len(descriptor_list):
                break
            sns.violinplot(y=X[:,descriptor_col_index],ax=ax[row,col],hue=y)
            ax[row,col].set_ylabel(f"{descriptor_list[descriptor_col_index]}")
            ax[row,col].set_title(f"Значения {descriptor_list[descriptor_col_index]}")
            descriptor_col_index +=1
def delete_eject(data,mode="iqr",iqr_param=1.5):
    descriptor_list = data.columns
    if mode == "iqr":
        for descriptor in descriptor_list:
            q1 = data[descriptor].quantile(0.25)
            q3 = data[descriptor].quantile(0.75)
            iqr = q3 - q1
            data = data[(data[descriptor] < q3 + iqr_param * iqr) & (data[descriptor] > q1 - iqr_param * iqr)]
    return data
def matrix_correlation_plot(data,level_index=0):
    descriptor_list = data.columns
    X = data.values
    y = data.index.get_level_values(level_index).tolist()    
    fig,ax = plt.subplots(len(descriptor_list),len(descriptor_list),figsize=(30,25))
    for i,feature_1 in enumerate(descriptor_list):
        for j,feature_2 in enumerate(descriptor_list):
            slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(X[:,i], X[:,j])
            sns.scatterplot(x=X[:,i],y=X[:,j],hue=y,ax=ax[i,j])
            ax[i,j].set_title(f"{feature_1} - {feature_2}, p-val = {round(p_value,2)}, R^2 = {round(r_value**2,2)}")
            ax[i,j].set_xlabel(f"{feature_1}")
            ax[i,j].set_ylabel(f"{feature_2}")
    plt.tight_layout()
def PCA_array(data):
    X = data.values
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    return X_pca
def multi_corr_test(data):
    descriptor_list = data.columns
    X = data.values    
    X_multicol_test = add_constant(X)
    VIFs = pd.DataFrame()
    VIFs['Variable'] = descriptor_list
    VIFs['VIF'] = [variance_inflation_factor(X_multicol_test, i) for i in range(X_multicol_test.shape[1]-1)]
    print(VIFs)